In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    setting = yaml.load(yaml_file)

path = make_path_dict(setting)

In [ ]:
w = pd.read_table(path["w.tsv"], index_col=0)

h = pd.read_table(path["h.tsv"], index_col=0)

w.columns.name = "Factor"

h.columns.name = setting["sample_alias"]

In [ ]:
for w_or_h in ("w", "h"):

    if w_or_h == "w":

        df = w

        normalization_axis = 1

        cluster_axis = 0

    elif w_or_h == "h":

        df = h

        normalization_axis = 0

        cluster_axis = 1

    w_or_h_directory_path = path["{}/".format(w_or_h)]

    ccal.plot_heat_map(
        df,
        normalization_axis=normalization_axis,
        normalization_method="-0-",
        cluster_axis=cluster_axis,
        title="NMF K={} {}".format(setting["nmf_k"], w_or_h.title()),
        xaxis_title=df.columns.name,
        yaxis_title=df.index.name,
        html_file_path="{}/heat_map.html".format(w_or_h_directory_path),
    )

In [ ]:
for element_type in ("feature", "sample"):

    elements_to_peek = setting["{}s_to_peek".format(element_type)]

    if elements_to_peek is not None:

        element_alias = setting["{}_alias".format(element_type)]

        if element_type == "feature":

            w_or_h = "w"

            df = w.loc[
                (
                    i
                    for i in w.index
                    if any(element in i for element in elements_to_peek)
                ),
                :,
            ]

            xaxis_title = "Factor"

            yaxis_title = element_alias

        elif element_type == "sample":

            w_or_h = "h"

            df = h.loc[
                :,
                (
                    i
                    for i in h.columns
                    if any(element in i for element in elements_to_peek)
                ),
            ]

            xaxis_title = element_alias

            yaxis_title = "Factor"

        w_or_h_directory_path = path["{}/".format(w_or_h)]

        ccal.plot_bubble_map(
            df,
            title="{} to Peek".format(element_alias),
            xaxis_title="Factor",
            yaxis_title=element_alias,
            html_file_path="{}/bubble_map.html".format(w_or_h_directory_path),
        )

In [ ]:
for w_or_h in ("w", "h"):

    if w_or_h == "w":

        df = w.T

    elif w_or_h == "h":

        df = h

    df_signature = pd.DataFrame(index=df.index, columns=df.columns)

    for factor_name, factor_values in df.iterrows():

        low_elements, high_elements = ccal.select_series_low_and_high_index(
            factor_values,
            title="NMF K={} {} {} Signature".format(
                setting["nmf_k"], w_or_h.title(), factor_name
            ),
        )

        df_signature.loc[factor_name, low_elements] = "Low"

        df_signature.loc[factor_name, high_elements] = "High"

    if w_or_h == "w":

        df_signature = df_signature.T

    df_signature.to_csv(path["{}|signature.tsv".format(w_or_h)], sep="\t")

In [ ]:
raw = pd.read_table(setting["feature_x_sample_file_path"], index_col=0)

signal = pd.read_table(path["feature_x_sample.context_signal.tsv"], index_col=0)

signal.fillna(0, inplace=True)

for w_or_h in ("w", "h"):

    if w_or_h == "w":

        targets = w.T

        df_raw = raw.T

        df_signal = signal.T

    elif w_or_h == "h":

        targets = h

        df_raw = raw

        df_signal = signal

    match_directory_path = path["{}|match/".format(w_or_h)]

    ccal.make_match_panels(
        targets,
        {
            "signal": {"df": df_signal, "data_type": "continuous"},
            "raw": {"df": df_raw, "data_type": "continuous"},
        },
        n_job=setting["max_n_job"],
        n_required_for_match_function=setting["n_required_for_match_function"],
        extreme_feature_threshold=setting["extreme_feature_threshold"],
        plot_std=setting["plot_std"],
        directory_path=match_directory_path,
    )